In [1]:
from leeq.utils.ai.experiment_generation.experiment_generation import generate_experiment
from leeq.utils.ai.experiment_generation.pulse_sequences import generate_pulse_sequences


In [2]:
description = """
Please measure the T1 time of the qubit in the following way:
1. Prepare the qubit to the excited state by applying a pi pulse.
2. Delay a time t, sweep time t 
3. Measure the qubit after the decay time.
4. fit the qubit excited state population versus time t.
5. Plot the excited state population versus time t both the data points and the fitted results. 

"""

In [3]:
generate_experiment(description)

Experiment Generation Progress:   0%|          | 0/5 [00:00<?, ?it/s]

'from labchronicle import log_and_record, register_browser_function\nfrom leeq import Experiment, Sweeper, SweepParametersSideEffectFactory, basic_run\nfrom leeq.core.primitives.logical_primitives import LogicalPrimitiveBlock\nfrom leeq.utils.compatibility import prims\nimport numpy as np\nfrom scipy.optimize import minimize\nimport matplotlib.pyplot as plt\n\nclass T1Experiment(Experiment):\n    """\n    Class for measuring the T1 time of a qubit by preparing it in the excited state,\n    allowing it to decay over time, and fitting the excited state population versus time.\n    """\n\n    @log_and_record\n    def run(self,\n            dut_qubit: Any,\n            collection_name: str = \'f01\',\n            mprim_index: int = 0,\n            start: float = 0.1,\n            stop: float = 1.0,\n            step: float = 0.1,\n            fit: bool = True) -> None:\n        """\n        Run the T1 experiment on a given qubit and analyze the results.\n\n        Parameters:\n        dut_

In [4]:
description = """

Please measure the T2 echo time of the qubit in the following way:
1. Prepare the qubit to the super position state by applying a pi/2 pulse.
2. Delay a time t, sweep time t 
3. apply a X gate to flip the qubit
4. Delay a same amount of time t
5. applying a pi/2 pulse.
6. Measure the qubit after the decay time.
7. fit the qubit excited state population versus time t.
8. Plot the excited state population versus time t both the data points and the fitted results. 

"""

In [5]:
generate_experiment(description)

Experiment Generation Progress:   0%|          | 0/5 [00:00<?, ?it/s]

'from labchronicle import log_and_record, register_browser_function\nfrom leeq import Experiment, Sweeper, SweepParametersSideEffectFactory, basic_run\nfrom leeq.core.primitives.logical_primitives import LogicalPrimitiveBlock\nfrom leeq.setups.built_in.setup_simulation_high_level import HighLevelSimulationSetup\nfrom leeq.utils.compatibility import prims\nfrom leeq.theory.fits import fit_exp_decay_with_cov\nimport numpy as np\nimport matplotlib.pyplot as plt\n\nclass T2EchoExperiment(Experiment):\n    """\n    Class for measuring the T2 echo time of a qubit by preparing it in a superposition state,\n    applying specific pulse sequences, and analyzing the decay of the excited state population over time.\n    """\n\n    @log_and_record\n    def run(self,\n            dut_qubit: Any,\n            t: float,\n            collection_name: str = \'f01\',\n            mprim_index: int = 0) -> Dict[str, float]:\n        """\n        Run the T2 echo experiment on a given qubit and analyze the r